<h1 align = 'center'> Mechine Learning Models

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linefinalar algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.metrics import classification_report 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Using TensorFlow backend.
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hanit\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
import torch

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

Only keeping the necessary columns.

In [4]:
data = pd.read_csv('Sentiment.csv', encoding= 'latin1')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [5]:
data.shape

(13871, 2)

Next, I am dropping the 'Neutral' sentiments as my goal was to only differentiate positive and negative tweets. After that, I am filtering the tweets so only valid texts and words remain.  Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [6]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [7]:
#data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [8]:
print('Positive: ', len(data[ data['sentiment'] == 'Positive']))
print('Negative: ', len(data[ data['sentiment'] == 'Negative']))
print('Neutral: ', len(data[ data['sentiment'] == 'Neutral']))

Positive:  2236
Negative:  8493
Neutral:  3142


In [9]:
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X,maxlen=100)

C:\Users\hanit\anaconda3\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [10]:
X.shape

(13871, 100)

In [11]:
data.sentiment.value_counts()

Negative    8493
Neutral     3142
Positive    2236
Name: sentiment, dtype: int64

In [12]:
data['sentiment'] = data.sentiment.apply(lambda x: 1 if x=='Positive' else (0 if x == 'Negative' else 2 ) )

In [13]:
data.sentiment.value_counts()

0    8493
2    3142
1    2236
Name: sentiment, dtype: int64

### Train test split

In [14]:
Y = data.sentiment
 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9293, 100) (9293,)
(4578, 100) (4578,)


# <span style='color:maroon'>Model Building</span>

### <span style='color:blue'>Multi Nominal Naivebayes</span>

**Creating model object**

In [15]:
# Multinomial Naive Bayes
MNB = MultinomialNB()

In [16]:
MNB.fit(X_train,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**Predictions**

In [17]:
# Predictions on train data
MNB_trainpreds =MNB.predict(X_train)
print("confusion_matrix on train data\n",confusion_matrix(Y_train,MNB_trainpreds))

confusion_matrix on train data
 [[2257  968 2459]
 [ 345  369  783]
 [ 440  305 1367]]


In [18]:
# classification report for Train data:
print(classification_report(y_true=Y_train,y_pred=MNB_trainpreds))

              precision    recall  f1-score   support

           0       0.74      0.40      0.52      5684
           1       0.22      0.25      0.24      1497
           2       0.30      0.65      0.41      2112

    accuracy                           0.43      9293
   macro avg       0.42      0.43      0.39      9293
weighted avg       0.56      0.43      0.45      9293



In [19]:
# Predictions on validation data
MNB_valpreds =MNB.predict(X_test)
print("confusion_matrix on train data\n",confusion_matrix(Y_test,MNB_valpreds))

confusion_matrix on train data
 [[1121  495 1193]
 [ 189  170  380]
 [ 224  151  655]]


In [20]:
MNB_valpreds

array([2, 1, 2, ..., 1, 0, 2], dtype=int64)

In [21]:
# classification report for validation data:
print(classification_report(y_true=Y_test,y_pred=MNB_valpreds))

              precision    recall  f1-score   support

           0       0.73      0.40      0.52      2809
           1       0.21      0.23      0.22       739
           2       0.29      0.64      0.40      1030

    accuracy                           0.43      4578
   macro avg       0.41      0.42      0.38      4578
weighted avg       0.55      0.43      0.44      4578



**Evalution**

In [22]:

# F1 Score for Train data:
print(f1_score(Y_train, MNB_trainpreds, average= 'weighted'))

0.4467280151342626


In [23]:
# F1 Score for Validation data:
print(f1_score(Y_test, MNB_valpreds, average="weighted"))

0.44251427000943966


### <span style='color:blue'>SGD Classifier</span>

In [24]:
from sklearn.linear_model import SGDClassifier

In [25]:
sgd_classifier = SGDClassifier(loss='hinge',
                                         penalty='l2',  alpha=0.0001, 
                                         l1_ratio=0.15,    fit_intercept=True, 
                                         max_iter=1000,    tol=0.001, shuffle=True, 
                                         verbose=0,        epsilon=0.1, n_jobs=None, 
                                         random_state=None, learning_rate='optimal', 
                                         eta0=0.0,          power_t=0.5, early_stopping=False, 
                                         validation_fraction=0.1, n_iter_no_change=5, 
                                         class_weight=None, warm_start=False, average=False)

In [26]:
## Fit on train data
sgd_classifier.fit(X=X_train, y= Y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [27]:
## Predict on train data
sgd_trainPreds = sgd_classifier.predict(X_train)

## Predict on validation data
sgd_valPreds = sgd_classifier.predict(X_test)

In [28]:
print("\nTRAIN DATA ACCURACY",accuracy_score(Y_train, sgd_trainPreds))
print("\nTrain data f1-score", f1_score(Y_train, sgd_trainPreds,average='weighted'))


TRAIN DATA ACCURACY 0.5136123964274185

Train data f1-score 0.4853558526700875


In [29]:
print("\nValidation DATA ACCURACY",accuracy_score(Y_test,sgd_valPreds))
print("\nValidation data f1-score", f1_score(Y_test, sgd_valPreds,average='weighted'))


Validation DATA ACCURACY 0.5192223678462211

Validation data f1-score 0.4877272185488563



### <span style='color:blue'>SVM</span>

In [30]:
SVC = SVC()

In [31]:
## Fit on train data
SVC.fit(X=X_train, y= Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

**Predictions**

In [32]:
## Predict on train data
svm_trainPreds = SVC.predict(X_train)

## Predict on validation data
svm_valPreds = SVC.predict(X_test)

In [33]:
### Train data accuracy


print("TRAIN Conf Matrix : \n", confusion_matrix(Y_train, svm_trainPreds))
print("\nTRAIN DATA ACCURACY",accuracy_score(Y_train, svm_trainPreds))
print("\nTrain data f1-score", f1_score(Y_train, svm_trainPreds,average='weighted'))

TRAIN Conf Matrix : 
 [[5678    0    6]
 [1444   49    4]
 [2048    6   58]]

TRAIN DATA ACCURACY 0.6225115678467664

Train data f1-score 0.4898710302596147


In [34]:
### Test data accuracy
print("Validation Conf Matrix : \n", confusion_matrix(Y_test, svm_valPreds))
print("\nValidation DATA ACCURACY",accuracy_score(Y_test,svm_valPreds))

Validation Conf Matrix : 
 [[2804    1    4]
 [ 725   11    3]
 [1011    5   14]]

Validation DATA ACCURACY 0.6179554390563565


### <span style='color:blue'>RandomForest</span>

In [35]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
Classifier_RF = RandomForestClassifier(criterion='entropy',n_estimators=10,max_depth=100,bootstrap = True,max_features = 'sqrt'
                                       ,min_samples_leaf=2,random_state=123,)

In [36]:
# Fit on training data
Classifier_RF.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=100, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [37]:
# Predictions on train data
RF_trainPreds = Classifier_RF.predict(X_train)
print("confusion_matrix on train data\n",confusion_matrix(Y_train,RF_trainPreds))

confusion_matrix on train data
 [[5515   76   93]
 [ 396 1036   65]
 [ 576   77 1459]]


In [38]:
# classification report for Train data:
print(classification_report(y_true=Y_train, y_pred=RF_trainPreds))

              precision    recall  f1-score   support

           0       0.85      0.97      0.91      5684
           1       0.87      0.69      0.77      1497
           2       0.90      0.69      0.78      2112

    accuracy                           0.86      9293
   macro avg       0.87      0.78      0.82      9293
weighted avg       0.87      0.86      0.86      9293



In [39]:
# F1 Score for Train data:
print(f1_score(Y_train, RF_trainPreds, average="macro"))

0.8200584281194284


In [40]:
# Predictions on Validation data
RF_valPreds = Classifier_RF.predict(X_test)
print("confusion_matrix on train data\n",confusion_matrix(Y_test,RF_valPreds))

confusion_matrix on train data
 [[2522   76  211]
 [ 508  163   68]
 [ 754   58  218]]


In [41]:
# classification report for Train data:
print(classification_report(y_true=Y_test, y_pred=RF_valPreds))

              precision    recall  f1-score   support

           0       0.67      0.90      0.77      2809
           1       0.55      0.22      0.31       739
           2       0.44      0.21      0.29      1030

    accuracy                           0.63      4578
   macro avg       0.55      0.44      0.46      4578
weighted avg       0.60      0.63      0.58      4578



In [42]:
# F1 Score for Train data:
print(f1_score(Y_test, RF_valPreds, average="macro"))

0.45508427904369664


In [43]:
prediction = pd.DataFrame(RF_valPreds)

In [44]:
prediction.to_csv("preds.csv")

In [45]:
prediction = prediction.rename(columns = {0:"Pred"})

In [46]:
X_test

array([[  0,   0,   0, ...,  21,  49,  49],
       [  0,   0,   0, ..., 989, 403, 563],
       [  0,   0,   0, ..., 122,   1,   3],
       ...,
       [  0,   0,   0, ..., 178, 875,   1],
       [  0,   0,   0, ..., 282, 789,   1],
       [  0,   0,   0, ..., 182, 767,  17]])

In [47]:
prediction.Pred.unique()

array([0, 2, 1], dtype=int64)

###  <span style='color:blue'>LSTM</span>


In [48]:
embed_dim = 256
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

C:\Users\hanit\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
C:\Users\hanit\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 256)          512000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               355152    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 867,743
Trainable params: 867,743
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9293, 100) (9293, 3)
(4578, 100) (4578, 3)


In [50]:
X_train

array([[   0,    0,    0, ...,  811,  161,  540],
       [   0,    0,    0, ...,   91,  539,    1],
       [   0,    0,    0, ..., 1992,   72, 1651],
       ...,
       [   0,    0,    0, ...,  976,   14,    1],
       [   0,    0,    0, ...,   37, 1478, 1424],
       [   0,    0,    0, ...,  223,   12,    1]])

In [51]:
Y_train

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]], dtype=uint8)

In [52]:
%%time

batch_size = 64

model.fit(X_train, Y_train, nb_epoch = 7, batch_size=batch_size, verbose = 2)

C:\Users\hanit\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until



Epoch 1/7
 - 23s - loss: 0.8389 - accuracy: 0.6427
Epoch 2/7
 - 24s - loss: 0.6844 - accuracy: 0.7050
Epoch 3/7
 - 25s - loss: 0.6165 - accuracy: 0.7416
Epoch 4/7
 - 24s - loss: 0.5685 - accuracy: 0.7633
Epoch 5/7
 - 24s - loss: 0.5256 - accuracy: 0.7825
Epoch 6/7
 - 24s - loss: 0.4885 - accuracy: 0.7994
Epoch 7/7
 - 24s - loss: 0.4510 - accuracy: 0.8136
Wall time: 2min 48s


In [53]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.94
acc: 0.65


In [54]:
X_validate.shape

(1500, 100)

In [55]:
pos_cnt, neg_cnt, pos_correct, neg_correct, nut_cnt, nut_correct = 0, 0, 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        elif np.argmax(Y_validate[x]) == 2:
            nut_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    elif np.argmax(Y_validate[x]) == 2:
        nut_cnt += 1
    else:
        pos_cnt += 1

In [56]:
print('pos_acc', round(pos_correct/pos_cnt*100, 2), '%')
print ('neg_acc', round(neg_correct/neg_cnt*100,2), "%")
print('Nuetral_acc', round(nut_correct/nut_cnt*100, 2), '%')

pos_acc 53.23 %
neg_acc 73.43 %
Nuetral_acc 40.0 %


###  <span style='color:blue'>NaiveBayes</span>

In [57]:
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS

In [58]:
data_nb = pd.read_csv('Sentiment.csv', encoding='latin1')

In [59]:
data = data_nb[['text', 'sentiment']]

In [60]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [61]:
# Splitting the dataset into train and test set
train, test = train_test_split(data,test_size = 0.1)

### Data cleaning of train data

In [62]:
tweets = []
stopwords_set = set(stopwords.words("english"))

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    tweets.append((words_cleaned,row.sentiment))
    

In [63]:
tweets[:2]

[(['note',
   'rand',
   'paul:',
   "don't",
   'pick',
   'fights',
   'with',
   'chris',
   'christie.'],
  'Neutral'),
 (['know',
   "he's",
   'sorta',
   'busy,',
   'but',
   'post-debate',
   'discussion',
   'just',
   "isn't",
   'the',
   'same',
   'without'],
  'Negative')]

In [64]:
           
tweets = []
stopwords_set = set(stopwords.words("english"))

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    tweets.append((words_cleaned,row.sentiment))

In [65]:
# Extracting word features
def get_words_in_tweets(tweets):
    all = []
    for (words, sentiment) in tweets:
        all.extend(words)
    return all

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features
w_features = get_word_features(get_words_in_tweets(tweets))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['containts(%s)' % word] = (word in document_words)
    return features

In [66]:
# Training the Naive Bayes classifier
training_set = nltk.classify.apply_features(extract_features,tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [67]:
test_pos = test[ test['sentiment'] == 'Positive']
test_pos = test_pos['text']
test_neg = test[ test['sentiment'] == 'Negative']
test_neg = test_neg['text']
test_nut = test[ test['sentiment'] == 'Neutral']
test_nut = test_nut['text']

In [68]:
neg_cnt = 0
pos_cnt = 0
nut_cnt = 0

for obj in test_neg: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'Negative'): 
        neg_cnt = neg_cnt + 1
        
for obj in test_pos: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'Positive'): 
        pos_cnt = pos_cnt + 1

for obj in test_nut: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'Neutral'): 
        nut_cnt = nut_cnt + 1
        
print('[Negative]: %s/%s '  % (neg_cnt,len(test_neg))) 
print('[Neutral]: %s/%s '  % (nut_cnt, len(test_nut)))
print('[Positive]: %s/%s '  % (pos_cnt, len(test_pos)))

[Negative]: 744/893 
[Neutral]: 98/304 
[Positive]: 65/191 


In [69]:
print("In Percentage: ")

print('[Negative]: ', round(neg_cnt/len(test_neg)*100,2))
print('[Neutral]: ', round(nut_cnt/len(test_nut)*100,2))
print('[Positive]:', round(pos_cnt/len(test_pos)*100,2)) 

In Percentage: 
[Negative]:  83.31
[Neutral]:  32.24
[Positive]: 34.03


In [70]:
X.shape

(13871, 100)

# Predictions on Unseen data

In [71]:
bramin_tweetDF = pd.read_csv('#BrahmanismRobsOBCReservationhashtagTweets.csv', encoding='latin1')
shame_tweetDF = pd.read_csv('#ShameOnNews18hashtagTweets.csv', encoding = 'latin1')

In [72]:
bramin_tweetDF.shape

(1790, 12)

#### Getting required columns

In [73]:
bramin_tweet = bramin_tweetDF[['text']]
shame_tweet = shame_tweetDF[['text']]

In [74]:
bramin_tweet.shape, shame_tweet.shape

((1790, 1), (1800, 1))

In [75]:
final1 = bramin_tweetDF
final2 = shame_tweetDF

In [77]:
final1.head(3)

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,TweetType
0,PunniyakotiMu,Tamilian by Language. Indian by Land. Humane b...,"Chennai, India",67,14,3949,2010-12-15 04:01:28,2020-07-21 03:01:18,711,OBC population is 52%.\nOBC Reservation is 27%...,[],1
1,Contractornesa5,NaN,NaN,425,132,2810,2019-05-31 03:23:24,2020-07-21 02:58:26,14,"@RangarajPandeyR In 1990, when VP Singh implem...",[],1
2,vcrasu,à®à®¿à®ªà®¿à®à®®à¯,NaN,158,5,1027,2020-06-04 08:45:04,2020-07-21 02:58:20,344,52% OBC's are Making two third of Hindus. Now ...,[],1


## UnseenData cleaning

In [78]:
def get_dataclean(tweet):
    tweet['text'] = tweet['text'].apply(lambda x: x.lower())
    tweet['text'] = tweet['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    for idx,row in tweet.iterrows():
        row[0] = row[0].replace('rt',' ')     
    return tweet

In [79]:
data1 = get_dataclean(bramin_tweet)
data2 = get_dataclean(shame_tweet)

C:\Users\hanit\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\hanit\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
#Test data shape
data2.shape

(1800, 1)

In [81]:
data1.head(3)

,text
0,obc population is 52\nobc reservation is 27\ne...
1,rangarajpandeyr in 1990 when vp singh implemen...
2,52 obcs are making two third of hindus now mod...


### Tokenization of Unseen data

In [82]:
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')

C:\Users\hanit\anaconda3\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [83]:
#Creating function for tokenizing

def Get_tokens(data):
    tokenizer.fit_on_texts(data['text'].values)
    g_token = tokenizer.texts_to_sequences(data['text'].values)
    g_pad = pad_sequences(g_token, maxlen=100)
    return g_pad

In [84]:
token1 = Get_tokens(data1)
token2 = Get_tokens(data2)

In [85]:
len(token1),  token1.shape

(1790, (1790, 100))

### Prediction
#### SVM

In [86]:
def Get_Model(token,model):
    prds = model.predict(token)
    return prds

In [87]:
def get_type(sentiment_score):
    senti = []
    for i in sentiment_score:
        if i == 1:
            senti.append("Positive")
        elif i == 0:
            senti.append("Negative")
        else:
            senti.append("Nuetral")
    return senti

In [88]:
df1 = Get_Model(token1, SVC)
df2 = Get_Model(token2, SVC)

In [89]:
df1 = get_type(df1)
df2 = get_type(df2)

In [90]:
df1[:10]

['Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative']

### RandomForest

In [92]:
df1_RFScore = Get_Model(token1, Classifier_RF)
df2_RFScore = Get_Model(token2, Classifier_RF)


In [93]:
df1_RF = get_type(df1_RFScore)
df2_RF = get_type(df2_RFScore)


In [94]:
df1_RF[:10]

['Negative',
 'Negative',
 'Negative',
 'Nuetral',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative']

### LSTM

In [95]:
def Get_LSTM(token,model):
    prds = model.predict(token)
    score = [np.argmax(i) for i in prds]
    return score

In [96]:
df1_lstmScore = Get_LSTM(token1, model)
df2_lstmScore = Get_LSTM(token2, model)


In [97]:
df1_lstm = get_type(df1_lstmScore)
df2_lstm = get_type(df2_lstmScore)


In [98]:
df1_lstm[:10]

['Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive']

**NaiveBayes**

In [99]:
def Get_NB_pred(data,model):
    res = []
    for obj in data.text:
        res.append(model.classify(extract_features(obj.split())))
    return res

In [100]:
result1 = Get_NB_pred(bramin_tweet, classifier)
result2 = Get_NB_pred(shame_tweet, classifier)
# result3 = Get_NB_pred(stepDown_tweet, classifier)

In [101]:
result1[:10]

['Negative',
 'Negative',
 'Neutral',
 'Negative',
 'Negative',
 'Neutral',
 'Negative',
 'Neutral',
 'Negative',
 'Negative']

In [102]:
len(result2)

1800

**Polarity**

In [103]:
import re
import textblob
from textblob import TextBlob

In [104]:
data1.head()

,text
0,obc population is 52\nobc reservation is 27\ne...
1,rangarajpandeyr in 1990 when vp singh implemen...
2,52 obcs are making two third of hindus now mod...
3,please suppo amp maximum \n\nbrahmanismrobso...
4,of 89 secretaries in modi govt there are just ...


In [105]:
data1['polarity'] = data1['text'].map(lambda text: TextBlob(text).sentiment.polarity)
data2['polarity'] = data2['text'].map(lambda text: TextBlob(text).sentiment.polarity)


C:\Users\hanit\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\hanit\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [106]:
#Defining function for converting sentiment type as Positive if sentiment score is (>0), neutral if sentiment score is (=0), 
#negative if sentiment score is (<0)

def Get_Polarity(_polarity):
    sentiment = "Positive"
    if _polarity > 0:
        sentiment= 'Positive'
                
    elif _polarity == 0:
        sentiment = 'Neutral'
            
    else:
        sentiment = 'Negative'
    
    return sentiment

In [108]:
polarity1 = data1.polarity.apply(Get_Polarity)
polarity2 = data2.polarity.apply(Get_Polarity)

In [109]:
polarity1[:10], type(polarity1)

(0    Negative
 1     Neutral
 2    Positive
 3     Neutral
 4     Neutral
 5    Positive
 6    Positive
 7     Neutral
 8    Negative
 9    Positive
 Name: polarity, dtype: object,
 pandas.core.series.Series)

In [110]:
polarity2.shape

(1800,)

## Final Report

In [111]:
final1

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,TweetType
0,PunniyakotiMu,Tamilian by Language. Indian by Land. Humane b...,"Chennai, India",67,14,3949,2010-12-15 04:01:28,2020-07-21 03:01:18,711,OBC population is 52%.\nOBC Reservation is 27%...,[],1
1,Contractornesa5,NaN,NaN,425,132,2810,2019-05-31 03:23:24,2020-07-21 02:58:26,14,"@RangarajPandeyR In 1990, when VP Singh implem...",[],1
2,vcrasu,à®à®¿à®ªà®¿à®à®®à¯,NaN,158,5,1027,2020-06-04 08:45:04,2020-07-21 02:58:20,344,52% OBC's are Making two third of Hindus. Now ...,[],1
3,ngdharan,NaN,NaN,91,94,5323,2012-03-29 07:34:49,2020-07-21 02:56:27,123,Please Support &amp; Maximum RT...\n\n#Brahman...,"[{'text': 'BrahmanismRobsOBCReservation', 'ind...",1
4,EzhilJerry,NaN,"Tamil Nadu, India",133,88,11929,2011-07-10 03:39:34,2020-07-21 02:55:27,3,"Of 89 secretaries in Modi govt, there are just...","[{'text': 'BrahmanismRobsOBCReservation', 'ind...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
1785,sabarivasandmk,Black reds are my life ð¤â¤ï¸ð¥,"Kanchipuram, Tamilnadu",206,68,1596,2020-05-05 07:50:45,2020-07-20 15:24:33,170,It took 65 years for my caste which comes int...,[],1
1786,sabarivasandmk,Black reds are my life ð¤â¤ï¸ð¥,"Kanchipuram, Tamilnadu",206,68,1596,2020-05-05 07:50:45,2020-07-20 15:24:31,124,Please Support &amp; Maximum RT...\n\n#Brahman...,"[{'text': 'BrahmanismRobsOBCReservation', 'ind...",1
1787,sabarivasandmk,Black reds are my life ð¤â¤ï¸ð¥,"Kanchipuram, Tamilnadu",206,68,1596,2020-05-05 07:50:45,2020-07-20 15:24:29,300,The Modi govt wants to include salary earnings...,[],1
1788,sabarivasandmk,Black reds are my life ð¤â¤ï¸ð¥,"Kanchipuram, Tamilnadu",206,68,1596,2020-05-05 07:50:45,2020-07-20 15:24:26,255,The data was obtained thru RTI by @asahcu. OBC...,[],1


In [112]:
def Final_report(data, svmSer, rfSer, lstmSer, NBPreds, PolarityPreds):
    report = pd.DataFrame()
    report = data
    report['SVM_Prediction'] = svmSer
    report['RF_Prediction'] = rfSer
    report['LSTM_Prediction'] = lstmSer
    report['NaiveBayes_Prediction'] = NBPreds
    report['Text_Blob_Prediction'] = PolarityPreds
    return report

In [113]:
report1 = Final_report(final1, df1, df1_RF,df1_lstm, result1, polarity1)
report2 = Final_report(final2, df2, df2_RF, df2_lstm, result2, polarity2)

In [114]:
report1.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,TweetType,SVM_Prediction,RF_Prediction,LSTM_Prediction,NaiveBayes_Prediction,Text_Blob_Prediction
0,PunniyakotiMu,Tamilian by Language. Indian by Land. Humane b...,"Chennai, India",67,14,3949,2010-12-15 04:01:28,2020-07-21 03:01:18,711,OBC population is 52%.\nOBC Reservation is 27%...,[],1,Negative,Negative,Negative,Negative,Negative
1,Contractornesa5,NaN,NaN,425,132,2810,2019-05-31 03:23:24,2020-07-21 02:58:26,14,"@RangarajPandeyR In 1990, when VP Singh implem...",[],1,Negative,Negative,Negative,Negative,Neutral
2,vcrasu,à®à®¿à®ªà®¿à®à®®à¯,NaN,158,5,1027,2020-06-04 08:45:04,2020-07-21 02:58:20,344,52% OBC's are Making two third of Hindus. Now ...,[],1,Negative,Negative,Negative,Neutral,Positive
3,ngdharan,NaN,NaN,91,94,5323,2012-03-29 07:34:49,2020-07-21 02:56:27,123,Please Support &amp; Maximum RT...\n\n#Brahman...,"[{'text': 'BrahmanismRobsOBCReservation', 'ind...",1,Negative,Nuetral,Negative,Negative,Neutral
4,EzhilJerry,NaN,"Tamil Nadu, India",133,88,11929,2011-07-10 03:39:34,2020-07-21 02:55:27,3,"Of 89 secretaries in Modi govt, there are just...","[{'text': 'BrahmanismRobsOBCReservation', 'ind...",1,Negative,Negative,Negative,Negative,Neutral


In [115]:
report2.shape

(1800, 17)

In [116]:
report1.to_csv(r'Report\#BrahmanismRobsOBCReservationhashtagTweetsPrediction.csv', index=False)
report2.to_csv(r'Report\#ShameOnNews18hashtagTweetsPrediction.csv', index=False)

# Batch4

In [ ]:
arrest_tweet = pd.read_csv('#ArrestKillersOfJayarajAndBennixhashtagTweets.csv', encoding = 'latin1')
madeIndia_tweet = pd.read_csv('#MadeInIndiahashtagTweets.csv', encoding='latin1')
modicare_tweet = pd.read_csv('#ModiCARES4PoorhashtagTweets.csv', encoding='latin1')
stop_tweet = pd.read_csv('#StopBhaashanTakeActionhashtagTweets.csv', encoding='latin1')

In [ ]:
arrest_tweet.shape

In [ ]:
arrest_tweet.head()

#### Getting required columns

In [ ]:
# arrest_tweet = arrest_tweet[['text','tweetcreatedts']]
# madeIndia_tweet = madeIndia_tweet[['text','tweetcreatedts']]
# modicare_tweet = modicare_tweet[['text', 'tweetcreatedts']]
# stop_tweet = stop_tweet[['text', 'tweetcreatedts']]

In [ ]:
final1 = arrest_tweet
final2 = madeIndia_tweet
final3 = modicare_tweet
final4 = stop_tweet

In [ ]:
final1.head()

In [ ]:
# arrest_tweet.head(10)

## UnseenData cleaning

In [ ]:
def get_dataclean(tweet):
    tweet['text'] = tweet['text'].apply(lambda x: x.lower())
    tweet['text'] = tweet['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    for idx,row in tweet.iterrows():
        row[0] = row[0].replace('rt',' ')     
    return tweet


In [ ]:
data1 = get_dataclean(arrest_tweet)
data2 = get_dataclean(madeIndia_tweet)
data3 = get_dataclean(modicare_tweet)
data4 = get_dataclean(stop_tweet)

In [ ]:
data2.text

## Tokenization of Unseen data

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')

In [ ]:
#Creating function for data cleaning
def Get_tokens(data):
    tokenizer.fit_on_texts(data['text'].values)
    g_token = tokenizer.texts_to_sequences(data['text'].values)
    g_pad = pad_sequences(g_token, maxlen=100)
    return g_pad

In [ ]:
token1 = Get_tokens(data1)
token2 = Get_tokens(data2)
token3 = Get_tokens(data3)
token4 = Get_tokens(data4)

In [ ]:
len(token1),  token1.shape

In [ ]:
token1[1]

## Predictions
### SVM

In [ ]:
# svm_unseenPred = SVC.predict(token1)

In [ ]:
def get_type(sentiment_score):
    senti = []
    for i in sentiment_score:
        if i == 1:
            senti.append("Positive")
        elif i == 0:
            senti.append("Negative")
        else:
            senti.append("Nuetral")
    return senti

In [ ]:
def Get_Model(token,model):
    prds = model.predict(token)
    return prds

In [ ]:
# series_pred_svm = only_model(token1, SVC)

In [ ]:
df1 = Get_Model(token1, SVC)
df2 = Get_Model(token2, SVC)
df3 = Get_Model(token3, SVC)
df4 = Get_Model(token4, SVC)

In [ ]:
# series_pred_svm = get_type(series_pred_svm)

In [ ]:
df1 = get_type(df1)
df2 = get_type(df2)
df3 = get_type(df3)
df4 = get_type(df4)

In [ ]:
# def Pred_testData(token,model, colName):
#     prds = model.predict(token)
#     df = pd.DataFrame(prds)
#     df_re = df.rename(columns={0: colName})
#     return df_re
           
    

### RandomForest

In [ ]:
# def Pred_testData(token,model, colName):
#     prds = model.predict(token)
#     df = pd.DataFrame(prds)
#     df_re = df.rename(columns={0: colName})
#     return df_re

In [ ]:
# series_pred_RF = only_model(token1, Classifier_RF)

In [ ]:
df1_RFScore = Get_Model(token1, Classifier_RF)
df2_RFScore = Get_Model(token2, Classifier_RF)
df3_RFScore = Get_Model(token3, Classifier_RF)
df4_RFScore = Get_Model(token4, Classifier_RF)

In [ ]:
df1_RF = get_type(df1_RFScore)
df2_RF = get_type(df2_RFScore)
df3_RF = get_type(df3_RFScore)
df4_RF = get_type(df4_RFScore)

In [ ]:
# df1_RF = Pred_testData(token1,Classifier_RF, colName='RF_Prediction')
# df2_RF = Pred_testData(token2, Classifier_RF, colName='RF_Prediction')
# df3_RF = Pred_testData(token3, Classifier_RF, colName='RF_Prediction')
# df4_RF = Pred_testData(token4, Classifier_RF, colName='RF_Prediction')

In [ ]:
# df1_RF.RF_Prediction.value_counts()

In [ ]:
# df1_RF['RF_Prediction'] = get_SetimentType(df1_RF['RF_Prediction'])
# df2_RF['RF_Prediction'] = get_SetimentType(df2_RF['RF_Prediction'])
# df3_RF['RF_Prediction'] = get_SetimentType(df3_RF['RF_Prediction'])
# df4_RF['RF_Prediction'] = get_SetimentType(df4_RF['RF_Prediction'])

In [ ]:
# df1_RF.RF_Prediction.value_counts()

### LSTM

In [ ]:
# def Get_LSTM(token,model):
#     prds = model.predict(token)
#     score = [np.argmax(i) for i in prds]
#     return score


In [ ]:
df1_lstmScore = Get_LSTM(token1, model)
df2_lstmScore = Get_LSTM(token2, model)
df3_lstmScore = Get_LSTM(token3, model)
df4_lstmScore = Get_LSTM(token4, model)

In [ ]:
df1_lstm = get_type(df1_lstmScore)
df2_lstm = get_type(df2_lstmScore)
df3_lstm = get_type(df3_lstmScore)
df4_lstm = get_type(df4_lstmScore)

### NaiveBayes

In [ ]:
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS

In [ ]:
def Get_NB_pred(data,model):
    res = []
    for obj in data.text:
        res.append(model.classify(extract_features(obj.split())))
    return res

In [ ]:
result1 = Get_NB_pred(arrest_tweet, classifier)
result2 = Get_NB_pred(madeIndia_tweet, classifier)
result3 = Get_NB_pred(modicare_tweet, classifier)
result4 = Get_NB_pred(stop_tweet, classifier)

In [ ]:
final1 = arrest_tweet
final2 = madeIndia_tweet
final3 = modicare_tweet
final4 = stop_tweet

In [ ]:
final1[:10]

**Polarity**

In [ ]:
import re
import textblob
from textblob import TextBlob

In [ ]:
data1.head(3)

In [ ]:
data1['polarity'] = data1['text'].map(lambda text: TextBlob(text).sentiment.polarity)
data2['polarity'] = data2['text'].map(lambda text: TextBlob(text).sentiment.polarity)
data3['polarity'] = data3['text'].map(lambda text: TextBlob(text).sentiment.polarity)
data4['polarity'] = data4['text'].map(lambda text: TextBlob(text).sentiment.polarity)

In [ ]:
#Defining function for converting sentiment type as Positive if sentiment score is (>0), neutral if sentiment score is (=0), 
#negative if sentiment score is (<0)

def Get_Polarity(_polarity):
    sentiment = "Positive"
    if _polarity > 0:
        sentiment= 'Positive'
                
    elif _polarity == 0:
        sentiment = 'Neutral'
            
    else:
        sentiment = 'Negative'
    
    return sentiment

In [ ]:
polarity1 = data1.polarity.apply(Get_Polarity)
polarity2 = data2.polarity.apply(Get_Polarity)
polarity3 = data3.polarity.apply(Get_Polarity)
polarity4 = data4.polarity.apply(Get_Polarity)

In [ ]:
polarity4[:10]

# FinalReport

In [ ]:
final1.head()

In [ ]:
def Final_report(data, svmSer, rfSer, lstmSer, NBPreds, PolarityPreds):
    report = pd.DataFrame()
    report = data
    report['SVM_Prediction'] = svmSer
    report['RF_Prediction'] = rfSer
    report['LSTM_Prediction'] = lstmSer
    report['NaiveBayes_Prediction'] = NBPreds
    report['Text_Blob_Prediction'] = PolarityPreds
    return report

In [ ]:
report1 = Final_report(final1, df1, df1_RF,df1_lstm, result1, polarity1)
report2 = Final_report(final2, df2, df2_RF, df2_lstm, result2, polarity2)
report3 = Final_report(final3, df3, df3_RF, df3_lstm, result3, polarity3)
report4 = Final_report(final4, df4, df4_RF, df4_lstm, result4, polarity4)

In [ ]:
report1.head()

In [ ]:
report1.to_csv(r'Report\#ArrestKillersOfJayarajAndBennixhashtagTweets_Report.csv', index=False)
report2.to_csv(r'Report\#MadeInIndiahashtagTweets_Report.csv', index=False)
report3.to_csv(r'Report\#ModiCARES4PoorhashtagTweets_Report.csv', index=False)
report4.to_csv(r'Report\#StopBhaashanTakeActionhashtagTweets_Report.csv', index=False)